# Similitud Semàntica de Textos

ELMO ==> EMBEDDING DE CARACTER ==> FILTRE CONVOLUCIONALS

BERT ==> TOKENITZAR PER OBTENIR N-GRAMS (DICCIONARI MÉS CURT)

HUGGINGFACE ==> MODELS PREENTRENATS

In [ ]:
from sentence_transformers import SentenceTransformer # EMBEDDING CONTEXTUAL FRASES AMB CÁLCUL DE DISTÀNCIA
import tensorflow as tf
import tensorflow_hub as hub
from tensorboard.plugins import projector
import numpy as np

from transformers import TFBertModel, BertTokenizer
from transformers import AutoModel, AutoTokenizer # tria automaticament el model i el tokenitzador

## EJEMPLO MODELO CALSSIFICACION A PARTIR DE BERT + CAPA EXTRA

In [ ]:
# Divideix les dades en conjunts d'entrenament i validació
train_sentences, val_sentences, train_labels, val_labels = train_test_split(
    sentences, labels, test_size=0.2, random_state=42
)

In [ ]:
# Base de dades vectorials per usar aquests models que generen vectors
def compute_similarity(sentence1, sentence2):
    # Carrega el model STS pre-entrenat
    model_name = 'stsb-roberta-base'
    model = SentenceTransformer(model_name)
    # Codifica les frases i calcula la similitud del cosinus
    embeddings1 = model.encode([sentence1])
    embeddings2 = model.encode([sentence2])
    similarity = cosine(embeddings1, embeddings2)
    return similarity
# Exemple d'ús
sentence1 = "I like cats"
sentence2 = "I love dogs"
similarity = compute_similarity(sentence1, sentence2)
print(f"Similitud: {similarity}")

In [ ]:
# Prepara les dades d'entrada en el format requerit per a NER
def prepare_input(corpus):
    sentences, labels = [], []
    for tagged_sentence in corpus:
        sentence, tag = zip(*tagged_sentence)
        sentences.append(" ".join(sentence))
        labels.append(list(tag))
    return sentences, labels
sentences, labels = prepare_input(corpus)

In [ ]:
model_name = "bert-base-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
# Tokenitza i converteix les frases al format d'entrada
train_encodings = tokenizer(train_sentences, truncation=True, padding=True)
val_encodings = tokenizer(val_sentences, truncation=True, padding=True)

In [ ]:
def convert_labels_to_ids(labels, label_map):
    label_ids = []
    for sent_labels in labels:
        label_ids.append([label_map[label] for label in sent_labels])
    return label_ids
# Defineix la correspondència d'etiquetes. Ordena-ho per tenir O abans. I després B abans que I.
labels_set = set(sum(labels, []))
label_list = sorted(labels_set, key=lambda e: "0" if e == "O" else e[2]+e[0])
label_map = {label: i for i, label in enumerate(label_list)}
# Converteix les etiquetes a IDs d'etiquetes
train_label_ids = convert_labels_to_ids(train_labels, label_map)
val_label_ids = convert_labels_to_ids(val_labels, label_map)

In [ ]:
train_label_ids = tf.keras.preprocessing.sequence.pad_sequences(
    train_label_ids, padding="post", truncating="post", maxlen=tokenizer.model_max_length)
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_label_ids
))
val_label_ids = tf.keras.preprocessing.sequence.pad_sequences(
    val_label_ids, padding="post", truncating="post", maxlen=tokenizer.model_max_length)
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_label_ids
))